## Generating Numpy arrays of data (Preprocessing)

In [ ]:
import numpy as np

data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

labelMap = {"Wine": 0, "Beer": 1, "Whiskey": 2}
inverseLabelMap = {labelMap[key]: key for key in labelMap}

y = np.array([labelMap[row[-1]] for row in data])
X = np.array([row[:-1] for row in data], dtype=float)

## Creating the GINI impurity calculation function

In [28]:
def giniImpurity(labels):
  counter = {key: 0 for key in list(set(labels))}
  for i in labels:
    counter[i] += 1
  return 1 - sum(((count / len(labels)) ** 2) for count in counter.values())

## Make the bestSplit function to identify the optimal feature and threshold to minimize weighted gini impurity of the left and right tree branches hence generated

In [29]:
def bestSplit(featureMatrix, labelArray):
  minGINI = 1
  bestThreshold = None
  bestFeature = None

  for feature in range(len(featureMatrix[0])):

    thresholds = np.unique(featureMatrix[:, feature])

    for threshold in thresholds:

      leftLabels = labelArray[featureMatrix[:, feature] <= threshold]
      rightLabels = labelArray[featureMatrix[:, feature] > threshold]

      if len(leftLabels) == 0 or len(rightLabels) == 0:
        continue

      giniLeft = giniImpurity(leftLabels)
      giniRight = giniImpurity(rightLabels)

      weightedGINI = (len(leftLabels) / len(labelArray)) * giniLeft + (len(rightLabels) / len(labelArray)) * giniRight

      if weightedGINI < minGINI:
        minGINI = weightedGINI
        bestThreshold = threshold
        bestFeature = feature

  return bestFeature, bestThreshold

## Making the decision binary tree recursively by using the bestSplit function to make branches

In [38]:
class Node:
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value # Only for leaf nodes

def mostCommonLabel(labels): # Find mode of the set of labels
  counter = {key: 0 for key in list(set(labels))}
  for i in labels:
    counter[i] += 1
  return max(counter, key=counter.get)

def buildTree(featureMatrix, labelArray, depth=0, maxDepth=3, minSamplesToSplit=2):
  if len(set(labelArray)) == 1 or len(labelArray) < minSamplesToSplit or depth >= maxDepth:
      leafValue = mostCommonLabel(labelArray)
      return Node(value=leafValue)

  bestFeature, bestThreshold = bestSplit(featureMatrix, labelArray)

  if bestFeature is None:
    leafValue = mostCommonLabel(labelArray)
    return Node(value=leafValue)

  leftIndices = featureMatrix[:, bestFeature] <= bestThreshold
  rightIndices = ~leftIndices

  leftFeatureMatrix, leftLabelArray = featureMatrix[leftIndices], labelArray[leftIndices]
  rightFeatureMatrix, rightLabelArray = featureMatrix[rightIndices], labelArray[rightIndices]

  leftNode = buildTree(featureMatrix=leftFeatureMatrix, labelArray=leftLabelArray, depth=depth+1)
  rightNode = buildTree(featureMatrix=rightFeatureMatrix, labelArray=rightLabelArray, depth=depth+1)

  return Node(
      feature_index=bestFeature,
      threshold=bestThreshold,
      left=leftNode,
      right=rightNode
  )

## We implement prediction function by traversing the tree recursively to predict the Label of a input record

In [39]:
def predict(node, record):
  if node.value is not None: # Node is a leaf node
    return node.value
  if record[node.feature_index] <= node.threshold:
    return predict(node.left, record)
  else:
    return predict(node.right, record)

## Evaluate the decision tree model with the training and test datasets

In [ ]:
# We build the model using the functions

decisionTree = buildTree(featureMatrix=X, labelArray=y)

# Evaluating on training set

error = 0
for record in data:
  prediction = predict(decisionTree, record[:-1])
  print("Prediction for : ", record[:-1], " is : ", inverseLabelMap[prediction])
  print("Actual Label : ", record[-1])
  error += (0 if inverseLabelMap[prediction] == record[-1] else 1)
print("Error Percentage : ", error/len(data) * 100, "%")

# Evaluating on test set

test_data = [
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
]

answers = ["Beer", "Whiskey", "Wine"]

error = 0
for i,record in enumerate(test_data):
  prediction = predict(decisionTree, record)
  print("Prediction for : ", record, " is : ", inverseLabelMap[prediction])
  print("Actual Label : ", answers[i])
  error += (0 if inverseLabelMap[prediction] == answers[i] else 1)
print("Error Percentage : ", error/len(data) * 100, "%")

Prediction for :  [12.0, 1.5, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [5.0, 2.0, 0]  is :  Beer
Actual Label :  Beer
Prediction for :  [40.0, 0.0, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [13.5, 1.2, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [4.5, 1.8, 0]  is :  Beer
Actual Label :  Beer
Prediction for :  [38.0, 0.1, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [11.5, 1.7, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [5.5, 2.3, 0]  is :  Beer
Actual Label :  Beer
Error Percentage :  0.0 %
Prediction for :  [6.0, 2.1, 0]  is :  Wine
Actual Label :  Beer
Prediction for :  [39.0, 0.05, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [13.0, 1.3, 1]  is :  Wine
Actual Label :  Wine
Error Percentage :  12.5 %


## Visualize the decision tree better for debigging purposes

In [42]:
def print_tree(node, spacing=""):
    if node.value is not None:
        print(spacing + "Predict", inverseLabelMap[node.value])
        return

    print(spacing + f"If feature[{node.feature_index}] <= {node.threshold}:")
    print_tree(node.left, spacing + "  ")
    print(spacing + f"Else:")
    print_tree(node.right, spacing + "  ")

print_tree(decisionTree)

If feature[0] <= 5.5:
  Predict Beer
Else:
  If feature[0] <= 13.5:
    Predict Wine
  Else:
    Predict Whiskey


## Optional : Use entropy criterion instead of gini

In [43]:
from math import log2
def entropyImpurity(labels):
  total = len(labels)
  if total == 0:
    return 0
  counter = {key: 0 for key in list(set(labels))}
  for i in labels:
    counter[i] += 1
  return -sum((count / total) * log2(count / total) for count in counter.values() if count > 0)

## We rewrite the bestSplit function to take input for which impurity criteria to use

In [44]:
def bestSplit(featureMatrix, labelArray, criterion="gini"):
  minGINI = 1
  bestThreshold = None
  bestFeature = None

  for feature in range(len(featureMatrix[0])):

    thresholds = np.unique(featureMatrix[:, feature])

    for threshold in thresholds:

      leftLabels = labelArray[featureMatrix[:, feature] <= threshold]
      rightLabels = labelArray[featureMatrix[:, feature] > threshold]

      if len(leftLabels) == 0 or len(rightLabels) == 0:
        continue

      if criterion == "gini":
        giniLeft = giniImpurity(leftLabels)
        giniRight = giniImpurity(rightLabels)
      else:
        giniLeft = entropyImpurity(leftLabels)
        giniRight = entropyImpurity(rightLabels)

      weightedGINI = (len(leftLabels) / len(labelArray)) * giniLeft + (len(rightLabels) / len(labelArray)) * giniRight

      if weightedGINI < minGINI:
        minGINI = weightedGINI
        bestThreshold = threshold
        bestFeature = feature

  return bestFeature, bestThreshold

## We now use this to build our decision tree :

In [46]:
class Node:
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value # Only for leaf nodes

def mostCommonLabel(labels): # Find mode of the set of labels
  counter = {key: 0 for key in list(set(labels))}
  for i in labels:
    counter[i] += 1
  return max(counter, key=counter.get)

def buildTree(featureMatrix, labelArray, depth=0, maxDepth=3, minSamplesToSplit=2):
  if len(set(labelArray)) == 1 or len(labelArray) < minSamplesToSplit or depth >= maxDepth:
      leafValue = mostCommonLabel(labelArray)
      return Node(value=leafValue)

  bestFeature, bestThreshold = bestSplit(featureMatrix, labelArray)

  if bestFeature is None:
    leafValue = mostCommonLabel(labelArray)
    return Node(value=leafValue)

  leftIndices = featureMatrix[:, bestFeature] <= bestThreshold
  rightIndices = ~leftIndices

  leftFeatureMatrix, leftLabelArray = featureMatrix[leftIndices], labelArray[leftIndices]
  rightFeatureMatrix, rightLabelArray = featureMatrix[rightIndices], labelArray[rightIndices]

  leftNode = buildTree(featureMatrix=leftFeatureMatrix, labelArray=leftLabelArray, depth=depth+1)
  rightNode = buildTree(featureMatrix=rightFeatureMatrix, labelArray=rightLabelArray, depth=depth+1)

  return Node(
      feature_index=bestFeature,
      threshold=bestThreshold,
      left=leftNode,
      right=rightNode
  )

## We evaluate our model made on the entropy criterion

In [48]:
from types import CoroutineType
# We build the model using the functions

decisionTree = buildTree(featureMatrix=features, labelArray=labels, )

# Evaluating on training set

error = 0
for record in data:
  prediction = predict(decisionTree, record[:-1])
  print("Prediction for : ", record[:-1], " is : ", inverseLabelMap[prediction])
  print("Actual Label : ", record[-1])
  error += (0 if inverseLabelMap[prediction] == record[-1] else 1)
print("Error Percentage : ", error/len(data) * 100, "%")

# Evaluating on test set

test_data = [
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
]

answers = ["Beer", "Whiskey", "Wine"]

error = 0
for i,record in enumerate(test_data):
  prediction = predict(decisionTree, record)
  print("Prediction for : ", record, " is : ", inverseLabelMap[prediction])
  print("Actual Label : ", answers[i])
  error += (0 if inverseLabelMap[prediction] == answers[i] else 1)
print("Error Percentage : ", error/len(test_data) * 100, "%")

Prediction for :  [12.0, 1.5, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [5.0, 2.0, 0]  is :  Beer
Actual Label :  Beer
Prediction for :  [40.0, 0.0, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [13.5, 1.2, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [4.5, 1.8, 0]  is :  Beer
Actual Label :  Beer
Prediction for :  [38.0, 0.1, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [11.5, 1.7, 1]  is :  Wine
Actual Label :  Wine
Prediction for :  [5.5, 2.3, 0]  is :  Beer
Actual Label :  Beer
Error Percentage :  0.0 %
Prediction for :  [6.0, 2.1, 0]  is :  Wine
Actual Label :  Beer
Prediction for :  [39.0, 0.05, 1]  is :  Whiskey
Actual Label :  Whiskey
Prediction for :  [13.0, 1.3, 1]  is :  Wine
Actual Label :  Wine
Error Percentage :  33.33333333333333 %


## Printing the tree to visualize


In [49]:
def print_tree(node, spacing=""):
    if node.value is not None:
        print(spacing + "Predict", inverseLabelMap[node.value])
        return

    print(spacing + f"If feature[{node.feature_index}] <= {node.threshold}:")
    print_tree(node.left, spacing + "  ")
    print(spacing + f"Else:")
    print_tree(node.right, spacing + "  ")

print_tree(decisionTree)

If feature[0] <= 5.5:
  Predict Beer
Else:
  If feature[0] <= 13.5:
    Predict Wine
  Else:
    Predict Whiskey
